In [1]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
import numpy as np

from transformers import pipeline
from diffusers.utils import load_image, make_image_grid

device = torch.device("cuda:0")

model_id = r"/datalake/homes/u_1401_00217454_x/.cache/torch/hub/checkpoints/genai/dpt-large"
image = load_image("/datalake/WMLA/data/d_1401/projects/d_00217454/code/dev/gen_ai_vision_lab/data_preparation/to_use/images/6022020103_180_image.jpg")

/datalake/WMLA/installation/W_Anaconda4Sigs/ana-1401-00217454/anaconda/envs/gai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
depth_estimator_weights = r"/datalake/homes/u_1401_00217454_x/.cache/torch/hub/checkpoints/genai/dpt-large"
sd_model_path = r"/datalake/homes/u_1401_00217454_x/.cache/torch/hub/checkpoints/genai/stable-diffusion-2-base"
cn_model_path = r"/datalake/homes/u_1401_00217454_x/.cache/torch/hub/checkpoints/genai/sd-controlnet-canny"

In [3]:
def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

depth_estimator = pipeline("depth-estimation", model = depth_estimator_weights)
depth_map = get_depth_map(image, depth_estimator).unsqueeze(0).half().to(device)

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at /datalake/homes/u_1401_00217454_x/.cache/torch/hub/checkpoints/genai/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [4]:
controlnet = ControlNetModel.from_pretrained(cn_model_path, torch_dtype=torch.float16, use_safetensors=True)
pipe = StableDiffusionControlNetPipeline.from_pretrained(sd_model_path, controlnet=controlnet, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to(device)

Loading pipeline components...: 100%|█████████████| 6/6 [00:07<00:00,  1.17s/it]


In [5]:
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

In [8]:
image = pipe("bird", image, num_inference_steps=20).images[0]
make_image_grid([original_image, canny_image, output], rows=1, cols=3)

NameError: name 'canny_image' is not defined

In [7]:
!nvidia-smi

Mon Mar 25 13:25:49 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   42C    P0    54W / 300W |   3987MiB / 32510MiB |      0%   E. Process |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A   